In [ ]:
# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# ============================================================================
# Cell 2: Dependencies and Configuration
# ============================================================================

import gradio as gr
import json
import csv
import datetime
from pathlib import Path
from typing import List, Dict, Any, Optional
import yaml

# Create necessary directories
outs_dir = Path("outs")
outs_dir.mkdir(exist_ok=True)
logs_dir = outs_dir / "logs"
logs_dir.mkdir(exist_ok=True)
presets_dir = Path("configs/presets")
presets_dir.mkdir(parents=True, exist_ok=True)

print("✓ Directories created")
print(f"✓ Logs: {logs_dir}")
print(f"✓ Presets: {presets_dir}")

In [ ]:
# ============================================================================
# Cell 3: Prompt Presets System
# ============================================================================


class PromptPresetManager:
    """管理提示詞預設的載入、儲存與應用"""

    def __init__(self, presets_dir: Path):
        self.presets_dir = presets_dir
        self.presets = {}
        self.load_default_presets()

    def load_default_presets(self):
        """載入預設的提示詞模板"""
        default_presets = {
            "chat_casual": {
                "name": "輕鬆聊天",
                "system": "你是一個友善的AI助手，用輕鬆自然的語氣與用戶對話。",
                "category": "chat",
                "description": "適合日常對話的輕鬆語氣",
            },
            "chat_professional": {
                "name": "專業助手",
                "system": "你是一個專業的AI助手，提供準確、簡潔且有建設性的回答。",
                "category": "chat",
                "description": "適合工作場合的專業語氣",
            },
            "rag_research": {
                "name": "研究分析",
                "system": "你是一個研究助手。基於提供的資料來源，進行深入分析並提供有根據的答案。請標註引用來源。",
                "category": "rag",
                "description": "適合研究分析的RAG模式",
            },
            "rag_qa": {
                "name": "問答助手",
                "system": "你是一個問答助手。根據提供的文檔內容回答問題，如果文檔中沒有相關資訊，請明確說明。",
                "category": "rag",
                "description": "適合文檔問答的RAG模式",
            },
            "agent_coordinator": {
                "name": "協調者",
                "system": "你是一個任務協調者。分析任務需求，制定執行計畫，並協調多個專家角色完成任務。",
                "category": "agents",
                "description": "適合多代理協調的模式",
            },
            "game_master": {
                "name": "遊戲主持人",
                "system": "你是一個文字冒險遊戲的主持人。創造生動的場景描述，提供有趣的選擇，並根據玩家決定推進故事。",
                "category": "game",
                "description": "適合文字冒險遊戲",
            },
        }

        self.presets = default_presets
        self.save_presets_to_file()

    def save_presets_to_file(self):
        """儲存預設到檔案"""
        preset_file = self.presets_dir / "default_presets.yaml"
        with open(preset_file, "w", encoding="utf-8") as f:
            yaml.dump(self.presets, f, ensure_ascii=False, indent=2)

    def load_presets_from_file(self, filename: str):
        """從檔案載入預設"""
        preset_file = self.presets_dir / filename
        if preset_file.exists():
            with open(preset_file, "r", encoding="utf-8") as f:
                loaded_presets = yaml.safe_load(f)
                self.presets.update(loaded_presets)

    def get_presets_by_category(self, category: str = None) -> Dict[str, Dict]:
        """根據類別取得預設"""
        if category is None:
            return self.presets
        return {k: v for k, v in self.presets.items() if v.get("category") == category}

    def get_preset_choices(self, category: str = None) -> List[str]:
        """取得預設選項列表（用於 Gradio dropdown）"""
        presets = self.get_presets_by_category(category)
        return [f"{preset['name']} ({key})" for key, preset in presets.items()]

    def apply_preset(self, preset_choice: str) -> str:
        """套用選定的預設"""
        if not preset_choice or preset_choice == "選擇預設...":
            return ""

        # Extract preset key from choice
        preset_key = preset_choice.split("(")[-1].rstrip(")")
        if preset_key in self.presets:
            return self.presets[preset_key]["system"]
        return ""


# Initialize preset manager
preset_manager = PromptPresetManager(presets_dir)
print("✓ Prompt presets loaded:")
for key, preset in preset_manager.presets.items():
    print(f"  - {preset['name']} ({preset['category']})")

In [ ]:
# ============================================================================
# Cell 4: Conversation History Management
# ============================================================================


class ConversationLogger:
    """管理對話記錄的儲存、載入與匯出"""

    def __init__(self, logs_dir: Path):
        self.logs_dir = logs_dir
        self.current_session = []
        self.session_metadata = {}

    def start_new_session(self, mode: str = "chat", model_info: str = ""):
        """開始新的對話會話"""
        self.current_session = []
        self.session_metadata = {
            "session_id": datetime.datetime.now().strftime("%Y%m%d_%H%M%S"),
            "start_time": datetime.datetime.now().isoformat(),
            "mode": mode,
            "model_info": model_info,
            "message_count": 0,
        }

    def log_message(self, role: str, content: str, metadata: Optional[Dict] = None):
        """記錄單一訊息"""
        message = {
            "timestamp": datetime.datetime.now().isoformat(),
            "role": role,
            "content": content,
            "metadata": metadata or {},
        }
        self.current_session.append(message)
        self.session_metadata["message_count"] = len(self.current_session)

    def log_conversation_turn(
        self,
        user_input: str,
        assistant_output: str,
        mode: str = "chat",
        sources: List[str] = None,
    ):
        """記錄一回合對話（用戶輸入 + 助手回應）"""
        # Log user message
        self.log_message("user", user_input)

        # Log assistant message with metadata
        assistant_metadata = {"mode": mode}
        if sources:
            assistant_metadata["sources"] = sources

        self.log_message("assistant", assistant_output, assistant_metadata)

    def save_session(self, filename: str = None) -> str:
        """儲存當前會話到檔案"""
        if not filename:
            filename = (
                f"session_{self.session_metadata.get('session_id', 'unknown')}.json"
            )

        filepath = self.logs_dir / filename

        session_data = {
            "metadata": self.session_metadata,
            "messages": self.current_session,
        }

        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(session_data, f, ensure_ascii=False, indent=2)

        return str(filepath)

    def load_session(self, filename: str):
        """從檔案載入會話"""
        filepath = self.logs_dir / filename
        if filepath.exists():
            with open(filepath, "r", encoding="utf-8") as f:
                session_data = json.load(f)
                self.session_metadata = session_data.get("metadata", {})
                self.current_session = session_data.get("messages", [])

    def export_to_markdown(self, filename: str = None) -> str:
        """匯出對話為 Markdown 格式"""
        if not filename:
            filename = (
                f"conversation_{self.session_metadata.get('session_id', 'export')}.md"
            )

        filepath = self.logs_dir / filename

        with open(filepath, "w", encoding="utf-8") as f:
            # Write metadata
            f.write(f"# 對話記錄\n\n")
            f.write(f"**會話ID**: {self.session_metadata.get('session_id', 'N/A')}\n")
            f.write(f"**開始時間**: {self.session_metadata.get('start_time', 'N/A')}\n")
            f.write(f"**模式**: {self.session_metadata.get('mode', 'N/A')}\n")
            f.write(
                f"**訊息數量**: {self.session_metadata.get('message_count', 0)}\n\n"
            )
            f.write("---\n\n")

            # Write messages
            for msg in self.current_session:
                role_display = "🧑 用戶" if msg["role"] == "user" else "🤖 助手"
                f.write(f"## {role_display}\n\n")
                f.write(f"{msg['content']}\n\n")

                # Add metadata if present
                if msg.get("metadata") and msg["metadata"]:
                    f.write(f"*時間: {msg['timestamp']}*\n\n")

        return str(filepath)

    def export_to_csv(self, filename: str = None) -> str:
        """匯出對話為 CSV 格式"""
        if not filename:
            filename = (
                f"conversation_{self.session_metadata.get('session_id', 'export')}.csv"
            )

        filepath = self.logs_dir / filename

        with open(filepath, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)

            # Write header
            writer.writerow(["timestamp", "role", "content", "mode", "sources"])

            # Write messages
            for msg in self.current_session:
                sources = msg.get("metadata", {}).get("sources", [])
                sources_str = "; ".join(sources) if sources else ""
                mode = msg.get("metadata", {}).get("mode", "")

                writer.writerow(
                    [msg["timestamp"], msg["role"], msg["content"], mode, sources_str]
                )

        return str(filepath)


# Initialize conversation logger
conv_logger = ConversationLogger(logs_dir)
print("✓ Conversation logger initialized")

In [ ]:
# ============================================================================
# Cell 5: Export Functions Implementation
# ============================================================================


def export_conversation(format_type: str, history: List[List[str]]) -> tuple:
    """匯出對話記錄到指定格式"""
    if not history:
        return None, "沒有對話記錄可以匯出"

    # Start new session and populate with history
    conv_logger.start_new_session()

    for turn in history:
        if (
            len(turn) >= 2 and turn[0] and turn[1]
        ):  # Ensure both user and assistant messages exist
            conv_logger.log_conversation_turn(turn[0], turn[1])

    try:
        if format_type == "JSON":
            filepath = conv_logger.save_session()
            success_msg = f"✓ 對話記錄已匯出至 JSON: {filepath}"

        elif format_type == "Markdown":
            filepath = conv_logger.export_to_markdown()
            success_msg = f"✓ 對話記錄已匯出至 Markdown: {filepath}"

        elif format_type == "CSV":
            filepath = conv_logger.export_to_csv()
            success_msg = f"✓ 對話記錄已匯出至 CSV: {filepath}"

        else:
            return None, f"不支援的匯出格式: {format_type}"

        return filepath, success_msg

    except Exception as e:
        return None, f"匯出失敗: {str(e)}"


def load_conversation_history(file_path: str) -> tuple:
    """載入對話記錄檔案"""
    if not file_path or not Path(file_path).exists():
        return [], "檔案不存在或路徑無效"

    try:
        conv_logger.load_session(Path(file_path).name)

        # Convert to Gradio chat history format
        history = []
        current_turn = [None, None]

        for msg in conv_logger.current_session:
            if msg["role"] == "user":
                if current_turn[0] is not None:  # Previous turn incomplete
                    history.append(current_turn)
                current_turn = [msg["content"], None]
            elif msg["role"] == "assistant":
                current_turn[1] = msg["content"]
                history.append(current_turn)
                current_turn = [None, None]

        return history, f"✓ 成功載入 {len(history)} 回合對話"

    except Exception as e:
        return [], f"載入失敗: {str(e)}"


print("✓ Export functions implemented")

In [ ]:
# ============================================================================
# Cell 6: UI Integration and Testing
# ============================================================================


def mock_chat_response(
    message: str, history: List[List[str]], system_prompt: str, mode: str
) -> tuple:
    """Mock chat response for testing (replace with actual LLM integration)"""

    # Simple mock responses based on mode
    if mode == "RAG":
        response = f"[RAG模式] 根據文檔搜尋，關於「{message[:20]}...」的相關資訊如下：\n\n這是一個模擬的RAG回應。\n\n來源: [1] 文檔A, [2] 文檔B"
    elif mode == "Agents":
        response = f"[Agents模式] 已分配任務給相關代理：\n1. 研究員：分析「{message[:20]}...」\n2. 規劃員：制定回應策略\n3. 撰寫員：產生最終回應\n\n這是協調後的結果。"
    elif mode == "Game":
        response = f"[遊戲模式] 🎮 你說：「{message[:30]}...」\n\n遊戲主持人回應：一個神秘的聲音在森林中迴響...\n\n你的選擇：\nA) 繼續前進\nB) 停下來聆聽\nC) 回頭離開"
    else:  # Chat mode
        response = f"我理解你的問題：「{message[:30]}...」\n\n這是一個友善的回應。根據當前的系統提示詞設定，我會用適當的語氣與你對話。"

    # Update history
    new_history = history + [[message, response]]

    # Log the conversation
    sources = ["文檔A", "文檔B"] if mode == "RAG" else []
    conv_logger.log_conversation_turn(message, response, mode, sources)

    return new_history, ""


def apply_preset_to_system(preset_choice: str) -> str:
    """Apply selected preset to system prompt"""
    return preset_manager.apply_preset(preset_choice)


def create_preset_demo_interface():
    """建立完整的預設與記錄管理介面"""

    with gr.Blocks(title="Prompt Presets & Conversation Logs") as demo:
        gr.Markdown("# 🔧 提示詞預設 & 對話記錄管理")

        with gr.Row():
            with gr.Column(scale=2):
                # Chat interface
                chatbot = gr.Chatbot(height=400, label="對話視窗")

                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="輸入你的訊息...", label="訊息輸入", scale=4
                    )
                    send_btn = gr.Button("發送", scale=1)

                with gr.Row():
                    clear_btn = gr.Button("清除對話")
                    mode_select = gr.Dropdown(
                        choices=["Chat", "RAG", "Agents", "Game"],
                        value="Chat",
                        label="模式",
                    )

            with gr.Column(scale=1):
                # Presets section
                gr.Markdown("### 📝 提示詞預設")

                preset_category = gr.Dropdown(
                    choices=["全部", "chat", "rag", "agents", "game"],
                    value="全部",
                    label="類別篩選",
                )

                preset_select = gr.Dropdown(
                    choices=["選擇預設..."] + preset_manager.get_preset_choices(),
                    value="選擇預設...",
                    label="選擇預設",
                )

                system_prompt = gr.Textbox(
                    label="系統提示詞", lines=8, placeholder="在此編輯系統提示詞..."
                )

                apply_preset_btn = gr.Button("套用預設", variant="primary")

                gr.Markdown("### 💾 對話記錄")

                # Export section
                export_format = gr.Dropdown(
                    choices=["JSON", "Markdown", "CSV"], value="JSON", label="匯出格式"
                )

                export_btn = gr.Button("匯出對話", variant="secondary")
                export_status = gr.Textbox(label="匯出狀態", interactive=False)

                # Load section
                load_file = gr.File(
                    label="載入對話記錄", file_types=[".json"], type="filepath"
                )
                load_btn = gr.Button("載入對話")
                load_status = gr.Textbox(label="載入狀態", interactive=False)

        # Event handlers
        def update_preset_choices(category):
            if category == "全部":
                choices = preset_manager.get_preset_choices()
            else:
                choices = preset_manager.get_preset_choices(category)
            return gr.Dropdown(choices=["選擇預設..."] + choices, value="選擇預設...")

        def handle_send(message, history, system_prompt, mode):
            if not message.strip():
                return history, ""
            return mock_chat_response(message.strip(), history, system_prompt, mode)

        def handle_export(format_type, history):
            filepath, status = export_conversation(format_type, history)
            return status

        def handle_load(file_path):
            if file_path:
                history, status = load_conversation_history(file_path)
                return history, status
            return [], "請選擇檔案"

        # Wire up events
        preset_category.change(
            update_preset_choices, inputs=[preset_category], outputs=[preset_select]
        )

        apply_preset_btn.click(
            apply_preset_to_system, inputs=[preset_select], outputs=[system_prompt]
        )

        send_btn.click(
            handle_send,
            inputs=[msg_input, chatbot, system_prompt, mode_select],
            outputs=[chatbot, msg_input],
        )

        msg_input.submit(
            handle_send,
            inputs=[msg_input, chatbot, system_prompt, mode_select],
            outputs=[chatbot, msg_input],
        )

        clear_btn.click(lambda: ([], ""), outputs=[chatbot, msg_input])

        export_btn.click(
            handle_export, inputs=[export_format, chatbot], outputs=[export_status]
        )

        load_btn.click(handle_load, inputs=[load_file], outputs=[chatbot, load_status])

    return demo


# Create and launch demo
demo = create_preset_demo_interface()
print("✓ UI interface created")

# For notebook testing, launch with share=False
if __name__ == "__main__":
    demo.launch(share=False, server_port=7854)

In [ ]:
# ============================================================================
# Cell 7: Smoke Test
# ============================================================================


def smoke_test_presets_and_logs():
    """驗證預設與記錄功能的基本運作"""

    print("🧪 開始 Smoke Test...")

    # Test 1: Preset management
    print("\n1. 測試預設管理...")
    test_presets = preset_manager.get_preset_choices("chat")
    assert len(test_presets) > 0, "Should have chat presets"
    print(f"✓ 找到 {len(test_presets)} 個聊天預設")

    # Test 2: Apply preset
    print("\n2. 測試預設套用...")
    test_choice = test_presets[0] if test_presets else "輕鬆聊天 (chat_casual)"
    applied_prompt = preset_manager.apply_preset(test_choice)
    assert applied_prompt != "", "Applied prompt should not be empty"
    print(f"✓ 成功套用預設: {applied_prompt[:50]}...")

    # Test 3: Conversation logging
    print("\n3. 測試對話記錄...")
    conv_logger.start_new_session("chat", "test-model")
    conv_logger.log_conversation_turn(
        "你好，這是測試訊息", "你好！我是AI助手，很高興為你服務。", "chat"
    )
    assert len(conv_logger.current_session) == 2, "Should have 2 messages"
    print(f"✓ 記錄了 {len(conv_logger.current_session)} 條訊息")

    # Test 4: Export functionality
    print("\n4. 測試匯出功能...")

    # Test JSON export
    json_path = conv_logger.save_session("test_session.json")
    assert Path(json_path).exists(), "JSON file should exist"
    print(f"✓ JSON 匯出成功: {json_path}")

    # Test Markdown export
    md_path = conv_logger.export_to_markdown("test_session.md")
    assert Path(md_path).exists(), "Markdown file should exist"
    print(f"✓ Markdown 匯出成功: {md_path}")

    # Test CSV export
    csv_path = conv_logger.export_to_csv("test_session.csv")
    assert Path(csv_path).exists(), "CSV file should exist"
    print(f"✓ CSV 匯出成功: {csv_path}")

    # Test 5: Load functionality
    print("\n5. 測試載入功能...")
    original_count = len(conv_logger.current_session)
    conv_logger.load_session("test_session.json")
    loaded_count = len(conv_logger.current_session)
    assert loaded_count == original_count, "Loaded session should match original"
    print(f"✓ 成功載入 {loaded_count} 條訊息")

    print(f"\n🎉 所有測試通過！")
    print(f"📁 記錄檔案位置: {logs_dir}")
    print(f"⚙️ 預設檔案位置: {presets_dir}")

    return True


# Run smoke test
smoke_test_result = smoke_test_presets_and_logs()
print(f"\n✅ Smoke Test 結果: {'通過' if smoke_test_result else '失敗'}")

In [ ]:
# ============================================================================
# Summary and Usage Notes
# ============================================================================

print(
    """
📋 **What we built:**
1. ✅ Prompt Presets 管理系統（6種預設類別）
2. ✅ 對話記錄的完整生命週期（記錄→儲存→匯出→載入）
3. ✅ 多格式匯出支援（JSON/Markdown/CSV）
4. ✅ Gradio UI 整合與預設選擇器
5. ✅ 會話元數據追蹤與時間戳記

🔧 **Key parameters:**
- presets_dir: configs/presets/ （預設儲存位置）
- logs_dir: outs/logs/ （記錄檔案位置）
- 支援格式: JSON, Markdown, CSV
- 類別篩選: chat, rag, agents, game

⚠️ **Pitfalls:**
1. 大量對話記錄可能影響檔案大小，建議定期清理
2. 匯出時確保有寫入權限到 outs/logs/ 目錄
3. JSON 格式包含完整元數據，適合程式解析
4. Markdown 格式適合人類閱讀，CSV 適合數據分析

📝 **When to use this:**
- 需要重複使用相同系統提示詞時
- 想要保存重要對話記錄時
- 需要分析對話模式或進行後續處理時
- 在不同模式間快速切換提示詞風格時
"""
)